In [2]:
from scatter.main import save, make_callable
from pprint import pprint


def sample(i: int, j: float = 5.0) -> list[float]:
    return i + j


save(sample)

name = "sample"

# my_func = _get_function(name)
# pprint(my_func.versioned_functions[my_func.current_version].model_dump())

# func = make_callable(name)
# print(func(2))


In [ ]:
# Measure time for 10000 calls

from time import perf_counter
import cloudpickle
import redis

NUM = 10_000


r = redis.Redis()
dumped_func = cloudpickle.dumps(sample)


# func_id = str(ulid.ULID())
func_id = "sample"
r.set(func_id, dumped_func)


start = perf_counter()
for _ in range(NUM):
    sample(42, 42)
print("Normal call:", perf_counter() - start)


start = perf_counter()
for _ in range(NUM):
    cloudpickle.loads(dumped_func)(42, 42)
print("Cloudpickle call:", perf_counter() - start)


start = perf_counter()
for _ in range(NUM):
    cloudpickle.loads(r.get(func_id))(42, 42)
print("Redis direct call:", perf_counter() - start)


start = perf_counter()
pipeline = r.pipeline()
for _ in range(NUM):
    pipeline.get(func_id)
all_funcs = pipeline.execute()
for func in all_funcs:
    cloudpickle.loads(func)(42, 42)
print("Redis pipelined call:", perf_counter() - start)


start = perf_counter()
for _ in range(NUM):
    make_callable(name)(42, 42)
print("Scatter:", perf_counter() - start)

In [3]:
# import redis

# r = redis.Redis()
# pipeline = r.pipeline()

# r.flushall()

True